In [114]:
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer

from pyspark.sql.functions import col, explode
import pandas as pd
import pyspark.sql.functions as func
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark import SparkContext, SQLContext

In [ ]:
! python --version

In [141]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.enableHiveSupport().getOrCreate()

conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '4g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '2'), ('spark.cores.max', '2'), ('spark.driver.memory','4g'),('spark.rpc.message.maxSize', '256')])

spark.sparkContext.stop()

spark = SparkSession.builder.config(conf=conf).getOrCreate()

#conf = SparkConf().setAppName("Collinear Points")

# Two ways you can access spark context from spark session
sc = spark._sc
sc2 = spark.sparkContext

#spark = SparkSession.builder.config("spark.driver.memory", "16G").getOrCreate()
#for later --conf spark.sql.catalogImplementation=hive 

#sc = spark._sc
#sc2 = spark.sparkContext

In [142]:
sc.getConf().getAll()

[('spark.driver.host', 'a-10-27-16-155.dynapool.vpn.nyu.edu'),
 ('spark.driver.memory', '4g'),
 ('spark.sql.warehouse.dir',
  'file:/Users/harlanhutton/Documents/NYU%201/Second%20Sem/Big%20Data/Labs/final/final-project-recommender-systers/spark-warehouse'),
 ('spark.executor.memory', '4g'),
 ('spark.executor.id', 'driver'),
 ('spark.rpc.message.maxSize', '256'),
 ('spark.app.id', 'local-1620075867084'),
 ('spark.executor.cores', '2'),
 ('spark.app.startTime', '1620075867018'),
 ('spark.app.name', 'Spark Updated Conf'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.cores.max', '2'),
 ('spark.driver.port', '61975'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [64]:
trainSample = spark.read.option("inferSchema", True).parquet('train_sample1.parquet')
testSample = spark.read.option("inferSchema", True).parquet('test_sample1.parquet')
trainSample.createOrReplaceTempView('trainSample')
testSample.createOrReplaceTempView('testSample')

In [65]:
valSample = spark.read.option("inferSchema", True).parquet('val_sample1.parquet')
valSample.createOrReplaceTempView('valSample')

In [66]:
indexer_obj_1 = StringIndexer(inputCol="user_id", outputCol="user_id_numer").setHandleInvalid("keep")
indexer_model_1 = indexer_obj_1.fit(trainSample)
indexer_df_1 = indexer_model_1.transform(trainSample)

indexer_obj_2 = StringIndexer(inputCol="track_id", outputCol="track_id_numer").setHandleInvalid("keep")
indexer_model_2= indexer_obj_2.fit(indexer_df_1)
indexer_df_2 = indexer_model_2.transform(indexer_df_1)

train_df = indexer_df_2.drop('user_id')
train_df= train_df.drop('track_id')


In [67]:
val_df_1 = indexer_model_1.transform(valSample)
val_df_2= indexer_model_2.transform(val_df_1)

val_df = val_df_2.drop('user_id')
val_df= val_df.drop('track_id')



In [69]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="user_id_numer", itemCol="track_id_numer", ratingCol="count",
          coldStartStrategy="drop")
model = als.fit(train_df)

#userRecs = model.recommendForAllUsers(10)

# Evaluate the model by computing the RMSE on the test data

# evaluator = RegressionEvaluator(metricName="rmse", labelCol="count",
#                                 predictionCol="prediction")
# rmse = evaluator.evaluate(predictions)

# print("Root-mean-square error = " + str(rmse))

# Generate top 10 song recommendations for each user

### ADJUST FOR 500 SONGS 

# Generate top 10 user recommendations for each song

#songRecs = model.recommendForAllItems(10)

In [ ]:
#### THIS WILL BE REMOVED ####

# Generate top 10 movie recommendations for a specified set of users
users = train_df.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)

# Generate top 10 user recommendations for a specified set of movies
# songs = df_dropped.select(als.getItemCol()).distinct().limit(3)
# songSubSetRecs = model.recommendForItemSubset(songs, 10)

In [ ]:
## MIGHT NOT NEED ##

predictions = model.transform(test_df)
predictions.orderBy('prediction', ascending=False)

In [70]:
# for each user, sort track ids by count
val_true = val_df.orderBy('count')

# flatten to group by user id and get list of true track ids
val_true_flatten = val_true.groupby('user_id_numer').agg(func.collect_list('track_id_numer').alias("track_id_numer"))

# add to dictionary
val_true_dict = val_true_flatten.collect()
val_true_dict = [{r['user_id_numer']: r['track_id_numer']} for r in val_true_dict]
val_true_dict = dict((key,d[key]) for d in val_true_dict for key in d)

{144345.0: [2576.0],
 257834.0: [5251.0, 1748.0],
 303341.0: [106998.0],
 279263.0: [5665.0],
 259239.0: [6919.0],
 231190.0: [106998.0],
 293951.0: [2662.0, 85544.0],
 159194.0: [4265.0],
 220558.0: [106998.0],
 321044.0: [18028.0],
 289159.0: [508.0],
 120784.0: [64175.0],
 314898.0: [21924.0, 28231.0],
 216365.0: [23.0, 69.0, 18.0],
 123501.0: [53918.0],
 223776.0: [71.0],
 147473.0: [7539.0, 25750.0],
 75511.0: [37293.0],
 304923.0: [22201.0],
 261625.0: [7574.0],
 295799.0: [40.0],
 157069.0: [106998.0],
 230320.0: [16226.0],
 292319.0: [9474.0],
 129140.0: [106998.0],
 111524.0: [9113.0],
 202200.0: [34735.0],
 265663.0: [975.0],
 290401.0: [29.0, 1.0, 9636.0],
 103678.0: [85936.0],
 213052.0: [12727.0],
 112524.0: [4.0],
 324257.0: [171.0],
 326013.0: [14874.0],
 309446.0: [63374.0],
 133623.0: [1013.0],
 270912.0: [22723.0],
 252645.0: [9436.0],
 76493.0: [5219.0],
 105880.0: [7848.0],
 305804.0: [11.0],
 193744.0: [5300.0],
 168872.0: [4099.0],
 320425.0: [5439.0],
 198823.0: 

In [ ]:
#https://stackoverflow.com/questions/59390481/how-to-implement-ranking-metrics-of-pyspark

In [71]:
val_preds = model.recommendForUserSubset(val_df,1)
val_preds_explode = val_preds.select(val_preds.user_id_numer,explode(val_preds.recommendations.track_id_numer))

val_preds_flatten = val_preds_explode.groupby('user_id_numer').agg(func.collect_list('col').alias("col"))

val_preds_dict = val_preds_flatten.collect()
val_preds_dict = [{r['user_id_numer']: r['col']} for r in val_preds_dict]
val_preds_dict = dict((key,d[key]) for d in val_preds_dict for key in d)

Evaluation Metric

In [144]:
labels_list = []

for user in val_preds_dict.keys():
    labels_list.append((val_preds_dict[user], [int(i) for i in val_true_dict[user]]))

labels = sc.parallelize(labels_list)
metrics = RankingMetrics(labels)
#print(metrics.meanAveragePrecision)
    
# add predicted items and true items as tuples to list

# parallelize list
# call ranking metrics on list 

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (a-10-27-16-155.dynapool.vpn.nyu.edu executor driver): org.apache.spark.SparkException: 
Error from python worker:
  dyld: Library not loaded: /System/Library/Frameworks/CoreFoundation.framework/Versions/A/CoreFoundation
    Referenced from: /Library/Frameworks/Python.framework/Versions/3.6/Resources/Python.app/Contents/MacOS/Python
    Reason: image not found
PYTHONPATH was:
  /Users/harlanhutton/opt/anaconda3/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip:/Users/harlanhutton/opt/anaconda3/lib/python3.8/site-packages/pyspark/python/lib/py4j-0.10.9-src.zip:/Users/harlanhutton/opt/anaconda3/lib/python3.8/site-packages/pyspark/jars/spark-core_2.12-3.1.1.jar
org.apache.spark.SparkException: EOFException occurred while reading

In [74]:
labels.collect()

[([28588], [5219]),
 ([31777], [63]),
 ([12756], [18967]),
 ([54627], [56086]),
 ([11827], [9113]),
 ([11827], [19998]),
 ([48884], [370]),
 ([28588], [7370]),
 ([28588], [9436]),
 ([28588], [18799]),
 ([31449], [412]),
 ([26092], [75104]),
 ([28588], [26177]),
 ([12756], [435]),
 ([11827], [395]),
 ([25176], [43290]),
 ([48884], [22201]),
 ([11827], [508]),
 ([11827], [10185]),
 ([48884], [5665]),
 ([54627], [14729]),
 ([48884], [106998]),
 ([12756], [29, 1, 9636]),
 ([54627], [5251, 1748]),
 ([11827], [4138]),
 ([28588], [6]),
 ([42403], [1869]),
 ([10472], [22614]),
 ([50608], [15983]),
 ([50608], [5477]),
 ([37291], [7539, 25750]),
 ([28588], [85936]),
 ([11827], [106998]),
 ([10896], [63374]),
 ([48884], [6544]),
 ([11827], [202]),
 ([1757], [7574]),
 ([36349], [19350]),
 ([31449], [4109]),
 ([50608], [10276]),
 ([9831], [4099]),
 ([31449], [81571]),
 ([49471], [2962]),
 ([9831], [3410]),
 ([31777], [490]),
 ([31340], [6275]),
 ([26092], [581]),
 ([12756], [139, 5840]),
 ([1757], 

In [80]:
userRecommended = model.recommendForUserSubset(val_df, 10).collect()
userRecommended_RDD = sc.parallelize(userRecommended).map(lambda k,v: (k, [x[1] for x in v]))

Exception ignored in: <function JavaModelWrapper.__del__ at 0x7fe3e01be160>
Traceback (most recent call last):
  File "/Users/harlanhutton/opt/anaconda3/lib/python3.8/site-packages/pyspark/mllib/common.py", line 137, in __del__
    self._sc._gateway.detach(self._java_model)
AttributeError: 'RankingMetrics' object has no attribute '_sc'
Exception ignored in: <function JavaModelWrapper.__del__ at 0x7fe3e01be160>
Traceback (most recent call last):
  File "/Users/harlanhutton/opt/anaconda3/lib/python3.8/site-packages/pyspark/mllib/common.py", line 137, in __del__
    self._sc._gateway.detach(self._java_model)
AttributeError: 'RankingMetrics' object has no attribute '_sc'


In [81]:
userRecommended_RDD.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 162.0 failed 1 times, most recent failure: Lost task 7.0 in stage 162.0 (TID 1837) (a-10-27-16-155.dynapool.vpn.nyu.edu executor driver): org.apache.spark.SparkException: 
Error from python worker:
  dyld: Library not loaded: /System/Library/Frameworks/CoreFoundation.framework/Versions/A/CoreFoundation
    Referenced from: /Library/Frameworks/Python.framework/Versions/3.6/Resources/Python.app/Contents/MacOS/Python
    Reason: image not found
PYTHONPATH was:
  /Users/harlanhutton/opt/anaconda3/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip:/Users/harlanhutton/opt/anaconda3/lib/python3.8/site-packages/pyspark/python/lib/py4j-0.10.9-src.zip:/Users/harlanhutton/opt/anaconda3/lib/python3.8/site-packages/pyspark/jars/spark-core_2.12-3.1.1.jar
org.apache.spark.SparkException: EOFException occurr

In [ ]:
userRecommended = model.recommendProductsForUsers(10).collect()
userRecommended_RDD = sc.parallelize(userRecommended)\
                         .map(lambda k,v: (k, [x[1] for x in v]))

# Get the actual usage
userMovies = validation_RDD.groupBy("new_uid")\
    .agg(F.collect_set("new_pgm")\
    .alias("actualVideos")).rdd

# join into a prediction vs actual RDD
predictionsAndLabels = userRecommended_RDD.join(userMovies).cache()

# Get the metrics
metricsRank = RankingMetrics(predictionsAndLabels.map(lambda r: r[1]))
tt_adv = time() - t0_adv

print "Time required : %.0f" % tt_adv
print "p5   %.8f" % metricsRank.precisionAt(5)
print "MAP  %.8f" % metricsRank.meanAveragePrecision
print "nDCG %.8f" % metricsRank.ndcgAt(5)

In [88]:
from pyspark import SparkContext
sc = SparkContext("local", "count app")
words = sc.parallelize (
   ["scala", 
   "java", 
   "hadoop", 
   "spark", 
   "akka",
   "spark vs hadoop", 
   "pyspark",
   "pyspark and spark"]
)
counts = words.count()
print("Number of elements in RDD -> %i" % (counts))

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by getOrCreate at <ipython-input-2-ae4c2209be47>:3 